## _*The EOH (Evolution of Hamiltonian) Algorithm*_

This notebook demonstrates how to use the `Qiskit Aqua` library to invoke the EOH algorithm and process the result.

Further information may be found for the algorithms in the online [Aqua documentation](https://qiskit.org/documentation/aqua/algorithms.html).

For this particular demonstration, we illustrate the `EOH` algorithm. First, two `Operator` instances we created are randomly generated Hamiltonians.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.transpiler import PassManager
from qiskit.aqua import run_algorithm, QuantumInstance
from qiskit.aqua.operator import Operator
from qiskit.aqua.algorithms import EOH
from qiskit.aqua.components.initial_states import Custom
from qiskit.aqua.input import EnergyInput

num_qubits = 2
temp = np.random.random((2 ** num_qubits, 2 ** num_qubits))
qubit_op = Operator(matrix=temp + temp.T)
temp = np.random.random((2 ** num_qubits, 2 ** num_qubits))
evo_op = Operator(matrix=temp + temp.T)

For EOH, we would like to evolve some initial state (e.g. the uniform superposition state) with `evo_op` and do a measurement using `qubit_op`. Below, we illustrate how such an example dynamics process can be easily prepared.

In [2]:
evo_time = 1
num_time_slices = 1
state_in = Custom(qubit_op.num_qubits, state='uniform')
eoh = EOH(qubit_op, state_in, evo_op, 'paulis', evo_time, num_time_slices)

We can then configure the quantum backend and execute our `EOH` instance:

In [3]:
backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

ret = eoh.run(quantum_instance)
print('The result is\n{}'.format(ret))

The result is
{'avg': (3.469125838650009-1.002992831056778e-16j), 'std_dev': 0.0}


The above programmatic approach can also be achieved via a declarative manner using a configuration configuration to specify the algorithm, components, backend etc. The operators, the main data for the algorithm, are supplied to it via an EnergyInput (an InputInstance type).

In [7]:
params = {
    'problem': {
        'name': 'eoh'
    },
    'algorithm': {
        'name': 'EOH',
        'num_time_slices': 1
    },
    'initial_state': {
        'name': 'CUSTOM',
        'state': 'uniform'
    },
    'backend': {
         'name': 'statevector_simulator',
         'provider': 'qiskit.BasicAer'
    }
}
algo_input = EnergyInput(qubit_op)
algo_input.add_aux_op(evo_op)

With all the necessary pieces prepared, we can then proceed to run the algorithm and examine the result. The run_algorithm takes the configuration dictionary and the input data.

In [9]:
ret = run_algorithm(params, algo_input)
print('The result is\n{}'.format(ret))

The result is
{'avg': (3.469125838650008-3.2364937588818136e-16j), 'std_dev': 0.0}
